In [59]:
import pandas as pd
import matplotlib.pyplot as plt
import cv2

import tensorflow as tf
from tensorflow import keras

from somefunc import *
%load_ext autoreload
%autoreload 2

images_path = "data/images"

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [47]:
df = pd.read_csv('data/cleaned_by_etienne.csv')

df["product_category_tree"] = df["product_category_tree"].apply(clean_categ)
df['path_file'] = df["uniq_id"].apply(get_path_file)
df['categ_0'] = df["product_category_tree"].apply(get_subcateg, args=(0))

df.drop(columns=['crawl_timestamp','product_url','pid', 'retail_price', 'discounted_price',
'image','is_FK_Advantage_product','product_rating','overall_rating','brand','product_specifications','uniq_id'],inplace=True)

categ_0_names = df.categ_0.unique()


In [60]:

images_list = get_img_list(df.path_file.values)
categ_0_list_num, categ_0_names= pd.factorize(df['categ_0'])
categ_0_list_hot =  tf.keras.utils.to_categorical(categ_0_list_num, num_classes=len(categ_0_names))


In [69]:
from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(images_list, categ_0_list_hot, test_size=0.1, random_state=42)

In [86]:
# ds = tf.data.Dataset.from_tensor_slices((images_list, categ_0_list_hot))
ds_train = tf.data.Dataset.from_tensor_slices((X_train, y_train))
ds_val = tf.data.Dataset.from_tensor_slices((X_val, y_val))

In [93]:
BATCH_SIZE=4
ds_train = ds_train.batch(BATCH_SIZE)
# .prefetch( tf.data.AUTOTUNE)
ds_val  = ds_val.batch(BATCH_SIZE)

In [94]:
image_batch,label_batch = next(iter(ds))
label_batch

<tf.Tensor: shape=(7,), dtype=float32, numpy=array([1., 0., 0., 0., 0., 0., 0.], dtype=float32)>

In [95]:
label_batch

<tf.Tensor: shape=(7,), dtype=float32, numpy=array([1., 0., 0., 0., 0., 0., 0.], dtype=float32)>

In [109]:
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras import layers
from tensorflow.keras.models import Model
IMG_SIZE = 224
base_model = EfficientNetB0(include_top=True, weights="imagenet")
base_model = Model(base_model.inputs, base_model.layers[-2].output)

model_in = layers.Input(shape=(IMG_SIZE, IMG_SIZE, 3))
temp_out = base_model(model_in)

model_out = layers.Dense(7,activation='softmax')(temp_out)

model_effnet = tf.keras.Model(model_in, model_out)

In [110]:
model_effnet.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping
es = EarlyStopping(mode='min', monitor='val_loss', patience=10, verbose=0)
rl = ReduceLROnPlateau(monitor='val_acc',factor=0.1,patience=5,verbose=1,mode="max",min_lr=0.0001)

In [112]:
history = model_effnet.fit(ds_train,
                            epochs = 50,
                            validation_data=ds_val,
                            # steps_per_epoch=50,
                            # validation_steps = 5
                            )

Epoch 1/50
237/237 [==============================] - 14s 57ms/step - loss: 0.8623 - accuracy: 0.7058 - val_loss: 1.3262 - val_accuracy: 0.6286
Epoch 2/50
237/237 [==============================] - 13s 55ms/step - loss: 0.5182 - accuracy: 0.8201 - val_loss: 1.8580 - val_accuracy: 0.5905
Epoch 3/50
237/237 [==============================] - 13s 55ms/step - loss: 0.4154 - accuracy: 0.8540 - val_loss: 1.0331 - val_accuracy: 0.7429
Epoch 4/50
237/237 [==============================] - 13s 55ms/step - loss: 0.2480 - accuracy: 0.9238 - val_loss: 1.3163 - val_accuracy: 0.7619
Epoch 5/50
237/237 [==============================] - 13s 54ms/step - loss: 0.2225 - accuracy: 0.9354 - val_loss: 1.9349 - val_accuracy: 0.7238
Epoch 6/50
237/237 [==============================] - 13s 55ms/step - loss: 0.2329 - accuracy: 0.9185 - val_loss: 1.1773 - val_accuracy: 0.7238
Epoch 7/50
237/237 [==============================] - 13s 55ms/step - loss: 0.1942 - accuracy: 0.9312 - val_loss: 1.7445 - val_accuracy:

KeyboardInterrupt: 

In [113]:
 model_effnet.save('models/ model_effnet01.h5')